In [1]:
import sys
import os

project_dir = '/home/nlp/achimoa/workspace/ModernHebrewBERT'
src_dir = os.path.join(project_dir, 'src')

os.chdir(project_dir)
print(os.getcwd())

if src_dir not in sys.path:
    sys.path.append(src_dir)
    print(f"Added {src_dir} to sys.path")

/home/nlp/achimoa/workspace/ModernHebrewBERT
Added /home/nlp/achimoa/workspace/ModernHebrewBERT/src to sys.path


In [3]:
from typing import Optional, cast
from omegaconf import DictConfig, OmegaConf
from omegaconf import OmegaConf as om

config_file = 'yamls/main/mosaic-bert-base-uncased.yaml'
with open("yamls/defaults.yaml") as f:
        default_cfg = om.load(f)
with open(config_file) as f:
    yaml_cfg = om.load(f)
cfg = om.merge(default_cfg, yaml_cfg)
cfg = cast(DictConfig, cfg)  # for type checking

print(f"Tokenizer name is {cfg.get('tokenizer_name', None)}")

Tokenizer name is bert-base-uncased


In [ ]:
from typing import Any, Callable, Dict, Iterator, List, Optional, Sequence, Union
from transformers import AutoTokenizer, PreTrainedTokenizer, PreTrainedTokenizerFast


Tokenizer = Union[PreTrainedTokenizer, PreTrainedTokenizerFast]

def build_tokenizer(
    om_tokenizer_config: DictConfig,
) -> Tokenizer:
    os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    resolved_om_tokenizer_config = om.to_container(om_tokenizer_config, resolve=True)
    tokenizer_kwargs = resolved_om_tokenizer_config.get(  # type: ignore
        "kwargs", {}
    )
    tokenizer_name = resolved_om_tokenizer_config["name"]  # type: ignore
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, **tokenizer_kwargs)

    # HuggingFace does not respect the model_max_length kwarg, and overrides it with
    # min(kwargs['model_max_length'], original_config['model_max_length']), so we
    # explicitly set it here
    tokenizer.model_max_length = tokenizer_kwargs.get(
        "model_max_length",
        int(1e30),
    )

    return tokenizer